In [58]:
from selenium import webdriver
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd

In [101]:
def getData(url):
    service = Service('C:/Users/alexa/Downloads/chromedriver-win64/chromedriver.exe')
    driver = webdriver.Chrome(service=service)

    driver.get(url)

    main_element = driver.find_element(By.ID, 'mainContent')

    scripts = main_element.find_elements(By.TAG_NAME, 'script')

    json_data = None

    for script in scripts:
        if 'application/ld+json' in script.get_attribute('type'):
            json_data = script.get_attribute('innerHTML')
            break

    driver.quit()

    if json_data:
        data = json.loads(json_data)
    
    else:
        print("pas de json")
        
    return data

In [61]:
def getMenu(data):
    menu = []
    for section in data['hasMenu'][0]['hasMenuSection']:
        for item in section['hasMenuItem']:
            menu.append([item['name'],item['description']])
        
    return menu

In [143]:
url ='https://www.opentable.com/r/hard-rock-cafe-paris?corrid=048b467e-411f-4336-b935-ecfaf102449e&avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&p=2&sd=2023-12-15T19%3A00%3A00'
data = getData(url)
print(data)

KeyboardInterrupt: 

In [120]:
print(data['hasMenu'])

[{'@type': 'Menu', 'name': 'Hard Rock Cafe Menu', 'hasMenuSection': [{'@type': 'MenuSection', 'name': 'STARTERS &amp; SHAREABLES', 'description': None, 'hasMenuItem': [{'@type': 'MenuItem', 'name': 'CLASSIC NACHOS', 'description': 'Crisp tortilla chips layered with ranch-style beans and a four-cheese sauce blend, topped with fresh pico de gallo, spicy jalapeños, melted cheddar and Monterey Jack cheese, pickled red onions and scallions, served with sour cream on the side.\nAdd Guacamole, Grilled Chicken, or Grilled Steak for an additional cost.'}, {'@type': 'MenuItem', 'name': 'ONE NIGHT IN BANGKOK SPICY SHRIMPTM', 'description': 'Crispy shrimp, tossed in a creamy, spicy sauce, topped with green onions, served on a bed of creamy coleslaw.'}, {'@type': 'MenuItem', 'name': 'WINGS', 'description': 'Our signature slow-roasted jumbo wings tossed with your choice of classic buffalo, sweet &amp; spicy tangy, or house-made barbecue sauce, served with celery and blue cheese dressing.'}, {'@type

In [121]:
getMenu(data)

[['CLASSIC NACHOS',
  'Crisp tortilla chips layered with ranch-style beans and a four-cheese sauce blend, topped with fresh pico de gallo, spicy jalapeños, melted cheddar and Monterey Jack cheese, pickled red onions and scallions, served with sour cream on the side.\nAdd Guacamole, Grilled Chicken, or Grilled Steak for an additional cost.'],
 ['ONE NIGHT IN BANGKOK SPICY SHRIMPTM',
  'Crispy shrimp, tossed in a creamy, spicy sauce, topped with green onions, served on a bed of creamy coleslaw.'],
 ['WINGS',
  'Our signature slow-roasted jumbo wings tossed with your choice of classic buffalo, sweet &amp; spicy tangy, or house-made barbecue sauce, served with celery and blue cheese dressing.'],
 ['BONELESS WINGS',
  'Crispy chicken thighs tossed with your choice of classic Buffalo, sweet &amp; spicy tangy, or house-made barbecue sauce, served with celery and blue cheese dressing.'],
 ['ALL-AMERICAN SLIDERS',
  'Three mini-burgers with melted American cheese, crispy onion ring and creamy 

In [ ]:
restaurants = pd.DataFrame(columns = ["nom","type","menu"])
if ('hasMenu' in data) and (data['hasMenu']!= []):

    typeCuisine = data['servesCuisine'].split(", ")

    resto = pd.DataFrame({'nom': data['name'], 'type': [typeCuisine], 'menu' : [getMenu(data)]})
    restaurants = pd.concat([restaurants, resto], ignore_index=True)
    restaurants

In [134]:
if ('hasMenu' in data) and (data['hasMenu']!= []):
    print("oui")
else :
    print("non")

oui


In [141]:
restaurants

,nom,type,menu
0,Hard Rock Cafe - Paris,[American],"[[CLASSIC NACHOS, Crisp tortilla chips layered..."
